[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/llm-agent-frameworks/crewai/simple-getting-started.ipynb)

# CrewAI and Weaviate Getting Started Example

This notebook shows you how industry-specific agents use web search and Weaviate blogs to track trends and uncover AI use cases across biomedical, healthcare, and finance.

## Import Libraries and Dependencies

At the time of creating this notebook, we're using `weaviate-client` version `4.15.4`, `crewai` and `crewai-tools` version `1.6.1` on a Python 3.13 version.

In [2]:
!pip install weaviate-client==4.15.4 crewai==1.6.1 crewai-tools==1.6.1

/opt/homebrew/Cellar/python@3.13/3.13.11/Frameworks/Python.framework/Versions/3.13/lib/python3.13/pty.py:95: DeprecationWarning: This process (pid=89859) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [1]:
import os
from dotenv import load_dotenv

from crewai_tools import WeaviateVectorSearchTool
import weaviate
from weaviate.classes.init import Auth

from crewai import Agent
from crewai_tools import WeaviateVectorSearchTool
from crewai import Task
from crewai import Crew, Process
from crewai_tools import SerperDevTool, WebsiteSearchTool

load_dotenv()


False

## Connect to Serper API and OpenAI

You can get 2,500 of free credits with Serper [here](https://serper.dev/)!

In [ ]:
os.environ["SERPER_API_KEY"] = "15a09aee29dd95d99842692b60a3f4de83049064"
#os.environ["OPENAI_API_KEY"] = ""

## Connect to Weaviate

Create a free sandbox cluster on [Weaviate Cloud](https://console.weaviate.cloud/)!

In [ ]:
WCD_CLUSTER_URL = "8yty7w5ws9mwt7aakuzja.c0.us-west3.gcp.weaviate.cloud"
WCD_CLUSTER_KEY = "RnIyT2p4d3hJSVhCQkZPaF8vWGVsM2EydHR2U05uenZMYUZQT1YxcktkSFA5MHdLZ0M2ZTdhMHpJSFdvPV92MjAw"

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WCD_CLUSTER_URL,
    auth_credentials=Auth.api_key(WCD_CLUSTER_KEY),
    headers={"X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY")}
)

print(client.is_ready())

## Connect to Existing Weaviate Collection

You'll want to connect to your existing Weaviate collection. If you don't have one already, you can follow this import notebook [here](https://github.com/weaviate/recipes/blob/main/integrations/Weaviate-Import-Example.ipynb)!

In [ ]:
weaviate_collection = client.collections.get("WeaviateBlogChunk")

## Initialize the Tools

### Weaviate Search Tool

In [ ]:
weaviate_tool = WeaviateVectorSearchTool(
    collection_name='WeaviateBlogChunk', 
    limit=4,
    alpha=0.75,
    weaviate_cluster_url=WCD_CLUSTER_URL,
    weaviate_api_key=WCD_CLUSTER_KEY,
)

### Web Search Tool

In [ ]:
search_tool = SerperDevTool()

## Create the Agents

### BioMed Marketing Agent

In [ ]:
BiomedicalMarketingAgent = Agent(
    role='Industry researcher focused on biomedical trends and their applications in AI',
    goal='Continuously track the latest biomedical advancements and identify how Weaviate’s features can support AI applications in biomedical research, diagnostics, and personalized medicine.',
    backstory='As a former biomedical product marketer turned AI strategist, you understand the complex language and regulatory landscape of biomedical innovation. With a keen eye on genomics, clinical research, and medical devices, it now leverages LLMs and vector search to explore how Weaviate’s capabilities can streamline scientific discovery and patient-centric campaigns.',
    llm="gpt-4o-mini",
    tools=[search_tool, weaviate_tool],
    verbose=True
)

### Healthcare Marketing Agent 

In [ ]:
HealthcareMarketingAgent = Agent(
    role='AI-savvy marketer specializing in healthcare systems, digital health, and patient engagement.',
    goal='Stay updated on healthcare policy shifts, digital health trends, and explore how Weaviate’s features can optimize workflows in hospital systems, EHR integration, and health communication.',
    backstory='Rooted in public health communications, this agent has evolved into a digital health consultant. You focus on how retrieval-augmented generation (RAG), semantic search, and hybrid models can be applied to solve healthcare-specific challenges—from patient triage to clinical support systems.',
    llm="gpt-4o-mini",
    tools=[search_tool, weaviate_tool],
    verbose=True
)

### Financial Marketing Agent

In [ ]:
FinancialMarketingAgent = Agent(
    role='Insight analyst exploring innovations in finance, wealth tech, and regulatory tech',
    goal='Monitor financial sector trends including AI in trading, compliance automation, and client advisory, and assess how Weaviate’s tools can enable cutting-edge financial applications.',
    backstory='With experience at a fintech startup and a background in capital markets, this agent specializes in using structured + unstructured data to surface insights for analysts and advisors. Now, it’s looking into how vector databases and LLMs can automate tasks like fraud detection, investor personalization, and market research.',
    llm="gpt-4o-mini",
    tools=[search_tool, weaviate_tool],
    verbose=True
)

## Define the Agents Tasks

### BioMed Agent Task

In [ ]:
biomed_agent_task = Task(
    description = """
        Conduct a thorough research about {weaviate_feature}
        Make sure you find any interesting and relevant information using the web and Weaviate blogs.
    """,
    expected_output = """
        Write an industry specific analysis of why this Weaviate feature would be useful for your industry of expertise.
    """,
    agent = BiomedicalMarketingAgent
)

### Healthcare Agent Task

In [ ]:
healthcare_agent_task = Task(
    description = """
        Conduct a thorough research about {weaviate_feature}
        Make sure you find any interesting and relevant information using the web and Weaviate blogs.
    """,
    expected_output = """
        Write an industry specific analysis of why this Weaviate feature would be useful for your industry of expertise.
    """,
    agent = HealthcareMarketingAgent
)

### Financial Agent Task

In [ ]:
financial_agent_task = Task(
    description = """
        Conduct a thorough research about {weaviate_feature}
        Make sure you find any interesting and relevant information using the web and Weaviate blogs.
    """,
    expected_output = """
        Write an industry specific analysis of why this Weaviate feature would be useful for your industry of expertise.
    """,
    agent = FinancialMarketingAgent
)

### List out the Weaviate Features

In [ ]:
weaviate_features = [
  {"weaviate_feature": "MUVERA"},
  {"weaviate_feature": "Multi-tenancy"},
  {"weaviate_feature": "Compliance"},
  {"weaviate_feature": "Hybrid Search"}
]

### Assemble the Crew

In [ ]:
blog_crew = Crew(
    agents=[BiomedicalMarketingAgent, HealthcareMarketingAgent, FinancialMarketingAgent],
    tasks=[biomed_agent_task, healthcare_agent_task, financial_agent_task],
    # verbose=True, # uncomment if you'd like to see the full execution from the kickoff
    # planning=True
)

## Query Time

In [ ]:
result = blog_crew.kickoff_for_each(inputs=weaviate_features)

print(result)